[69 BERT预训练](https://www.bilibili.com/video/BV1yU4y1E7Ns?p=2&vd_source=1d3a7b81d826789081d8b6870d4fff8e)

- word2vec将相同的预训练向量分配给同一个词，而不考虑词的上下文（如果有的话）。
- 形式上，任何词元x的上下文无关表示是函数，其仅将f(x)作为其输入。考虑到自然语言中丰富的多义现象和复杂的语义，上下文无关表示具有明显的局限性。
  - 例如，在“a crane is flying”（一只鹤在飞）和“a crane driver came”（一名吊车司机来了）的上下文中，“crane”一词有完全不同的含义；
  - 因此，同一个词可以根据上下文被赋予不同的表示。

针对其上下文不敏感的问题, 后面也陆续推出了更多的模型, 包括但不限于
- [TagLM language-model-augmented sequence tagger，语言模型增强的序列标记器）](/%E9%99%84%E5%BD%9500%20%E5%8F%82%E8%80%83%E6%96%87%E7%8C%AE/Semi-supervised%20sequence%20tagging%20with%20bidirectional%20language%20models.pdf)
- [CoVe（Context Vectors，上下文向量）](/%E9%99%84%E5%BD%9500%20%E5%8F%82%E8%80%83%E6%96%87%E7%8C%AE/NIPS-2017-learned-in-translation-contextualized-word-vectors-Paper.pdf)
- [ELMo（Embeddings from Language Models，来自语言模型的嵌入](/%E9%99%84%E5%BD%9500%20%E5%8F%82%E8%80%83%E6%96%87%E7%8C%AE/Deep%20contextualized%20word%20representations.pdf)
  - ELMo改进了六种自然语言处理任务的技术水平：情感分析、自然语言推断、语义角色标注、共指消解、命名实体识别和问答。

ELMo依旧不够通用，[GPT（Generative Pre Training，生成式预训练）](/%E9%99%84%E5%BD%9500%20%E5%8F%82%E8%80%83%E6%96%87%E7%8C%AE/Improving%20Language%20Understanding%20by%20Generative%20Pre-Training.pdf)能够一定程度上改进其水平，但GPT只能向前看（从左到右）。在“i went to the bank to deposit cash”（我去银行存现金）和“i went to the bank to sit down”（我去河岸边坐下）的上下文中，由于“bank”对其左边的上下文敏感，GPT将返回“bank”的相同表示，尽管它有不同的含义。

- [BERT（来自Transformers的双向编码器表示）](/%E9%99%84%E5%BD%9500%20%E5%8F%82%E8%80%83%E6%96%87%E7%8C%AE/BERT%20Pre-training%20of%20Deep%20Bidirectional%20Transformers%20for%20Language%20Understanding.pdf)结合了这两个方面的优点。它对上下文进行双向编码，并且对于大多数的自然语言处理任务 [Devlin et al., 2018]只需要最少的架构改变。通过使用预训练的Transformer编码器，BERT能够基于其双向上下文表示任何词元。在下游任务的监督学习过程中，BERT在两个方面与GPT相似。首先，BERT表示将被输入到一个添加的输出层中，根据任务的性质对模型架构进行最小的更改，例如预测每个词元与预测整个序列。其次，对预训练Transformer编码器的所有参数进行微调，而额外的输出层将从头开始训练。 下图描述了ELMo、GPT和BERT之间的差异。

<img src= "http://zh.d2l.ai/_images/elmo-gpt-bert.svg">

- BERT进一步改进了11种自然语言处理任务的技术水平，这些任务分为以下几个大类：
- （1）单一文本分类（如情感分析）、（2）文本对分类（如自然语言推断）、（3）问答、（4）文本标记（如命名实体识别）。
- 从上下文敏感的ELMo到任务不可知的GPT和BERT，它们都是在2018年提出的。概念上简单但经验上强大的自然语言深度表示预训练已经彻底改变了各种自然语言处理任务的解决方案。

### 输入表示
- 在自然语言处理中，有些任务（如情感分析）以单个文本作为输入，
- 而有些任务（如自然语言推断）以一对文本序列作为输入。BERT输入序列明确地表示单个文本和文本对。
- 当输入为单个文本时，BERT输入序列是特殊类别词元“<cls>”、文本序列的标记、以及特殊分隔词元“<sep>”的连结。
- 当输入为文本对时，BERT输入序列是“<cls>”、第一个文本序列的标记、“<sep>”、第二个文本序列标记、以及“<sep>”的连结。
- 我们将始终如一地将术语“BERT输入序列”与其他类型的“序列”区分开来。例如，一个BERT输入序列可以包括一个文本序列或两个文本序列。

为了区分文本对，根据输入序列学到的片段嵌入$\mathbf{e}_A$和$\mathbf{e}_B$分别被添加到第一序列和第二序列的词元嵌入中。对于单文本输入，仅使用$\mathbf{e}_A$。

下面的get_tokens_and_segments将一个句子或两个句子作为输入，然后返回BERT输入序列的标记及其相应的片段索引。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

/opt/conda/envs/ml/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/envs/ml/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# 这个函数允许输入单个句子和句子对
# 单个句子的时候，token_b可以为none
# 句子将以list(token)形式输入
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """获取输入序列的词元及其片段索引"""
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    # 0和1分别标记片段A和B
    # 这里的segment相当于一个标记符，0标记第一个句子，1标记第二个句子
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>']
        segments += [1] * (len(tokens_b) + 1)
    return tokens, segments

bert的一点创新就是位置编码是可学习的

<img src="http://zh.d2l.ai/_images/bert-input.svg">

In [3]:
class BERTEncoder(nn.Module):
    """BERT编码器"""
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768,
                 **kwargs):
        """
            vocab_size 词表大小
            num_hiddens 用于embedding的大小，也就是维度大小
            norm_shape layer_norm 层规范化
            ffn_num_input 前馈层的输入大小
            ffn_num_hiddens 前馈层的隐藏层大小
            num_heads 多头注意力机制需要使用的注意力层数量
            num_layers bert要使用的transformer的encoder数量
            dropout 前馈层是否使用dropout以及比例
            max_len 输入的位置编码的最大长度
            key_size，query_size，value_size 注意力层的k q v大小，k q大小必须一致，v随意，不过一般也会一样大
        """
        super(BERTEncoder, self).__init__(**kwargs)
        self.token_embedding = nn.Embedding(vocab_size, num_hiddens)
        self.segment_embedding = nn.Embedding(2, num_hiddens)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            # 这里是直接n个transformer的encoderblock
            self.blks.add_module(f"{i}", d2l.EncoderBlock(
                key_size, query_size, value_size, num_hiddens, norm_shape,
                ffn_num_input, ffn_num_hiddens, num_heads, dropout, True))
        # 在BERT中，位置嵌入是可学习的，因此我们创建一个足够长的位置嵌入参数
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len,
                                                      num_hiddens))

    def forward(self, tokens, segments, valid_lens):
        # 在以下代码段中，X的形状保持不变：（批量大小，最大序列长度，num_hiddens）
        X = self.token_embedding(tokens) + self.segment_embedding(segments)
        # 位置编码只需要拿到实际长度就行
        X = X + self.pos_embedding.data[:, :X.shape[1], :]
        for blk in self.blks:
            X = blk(X, valid_lens)
        return X

假设词表大小为10000，为了演示BERTEncoder的前向推断，让我们创建一个实例并初始化它的参数。

In [4]:
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024, 4
norm_shape, ffn_num_input, num_layers, dropout = [768], 768, 2, 0.2
encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape, ffn_num_input,
                      ffn_num_hiddens, num_heads, num_layers, dropout)

我们将tokens定义为长度为8的2个输入序列，其中每个词元是词表的索引。使用输入tokens的BERTEncoder的前向推断返回编码结果，其中每个词元由向量表示，其长度由超参数num_hiddens定义。此超参数通常称为Transformer编码器的隐藏大小（隐藏单元数）。

In [5]:
tokens = torch.randint(0, vocab_size, (2, 8))
print('tokens: ',tokens)
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])
encoded_X = encoder(tokens, segments, None)
encoded_X.shape

tokens:  tensor([[ 837, 6909, 3069, 6597,  183, 4804, 3197, 8514],
        [4555,  263, 3847, 6987, 7804, 5731, 5911, 5866]])


torch.Size([2, 8, 768])

In [6]:
encoded_X

tensor([[[-0.1843,  0.1508,  0.4777,  ..., -1.2083,  0.0782,  0.9432],
         [-0.4222, -0.0557,  1.2787,  ..., -1.2895, -0.8856, -1.2039],
         [-0.5091,  0.1737,  0.9595,  ..., -1.1155, -0.3339,  1.4115],
         ...,
         [ 0.9662, -0.4583,  1.7574,  ..., -1.6314, -1.6400, -0.6994],
         [-0.3512, -0.6939,  0.3962,  ..., -2.3855, -0.1926, -1.1189],
         [ 1.7111, -0.0748, -1.1593,  ..., -2.6879, -2.0826, -0.2087]],

        [[ 0.4218, -0.1173,  0.5914,  ..., -0.6605, -0.3040, -1.0521],
         [-1.6564, -0.2103,  0.4227,  ..., -1.5611,  0.2880, -0.3879],
         [-0.1985, -0.6325,  0.8929,  ..., -1.4288, -1.4854,  1.4637],
         ...,
         [ 1.1318,  0.4328,  1.5451,  ..., -1.3508, -1.3843, -0.1113],
         [ 1.1183, -1.5726,  0.5453,  ..., -1.4155, -0.5513,  0.0409],
         [ 1.1957, -0.1602, -1.5642,  ..., -2.6825, -2.9738,  0.9956]]],
       grad_fn=<NativeLayerNormBackward0>)

- 在这个预训练任务中，将随机选择15%的词元作为预测的掩蔽词元。要预测一个掩蔽词元而不使用标签作弊，一个简单的方法是总是用一个特殊的$“<mask>”$替换输入序列中的词元。然而，人造特殊词$元“<mask>”$不会出现在微调中。为了避免预训练和微调之间的这种不匹配，如果为预测而屏蔽词元（例如，在“this movie is great”中选择掩蔽和预测“great”），则在输入中将其替换为：

  - 80%时间为特殊的$“<mask>“$词元（例如，“this movie is great”变为“this movie is$<mask>$”；

  - 10%时间为随机词元（例如，“this movie is great”变为“this movie is drink”）；
  
    - 这种偶然的噪声鼓励BERT在其双向上下文编码中不那么偏向于掩蔽词元（尤其是当标签词元保持不变时）。 

  - 10%时间内为不变的标签词元（例如，“this movie is great”变为“this movie is great”）。



- 我们实现了下面的MaskLM类来预测BERT预训练的掩蔽语言模型任务中的掩蔽标记。
- 预测使用单隐藏层的多层感知机（self.mlp）。在前向推断中，它需要两个输入：
  - BERTEncoder的编码结果和用于预测的词元位置。输出是这些位置的预测结果。

In [7]:
class MaskLM(nn.Module):
    """BERT的掩蔽语言模型任务"""
    def __init__(self, vocab_size, num_hiddens, num_inputs=768, **kwargs):
        super(MaskLM, self).__init__(**kwargs)
        self.mlp = nn.Sequential(nn.Linear(num_inputs, num_hiddens),
                                 nn.ReLU(),
                                 nn.LayerNorm(num_hiddens),
                                 nn.Linear(num_hiddens, vocab_size))
        
    # position代表了哪些句子需要进行预测
    def forward(self, X, pred_positions):
        num_pred_positions = pred_positions.shape[1]
        pred_positions = pred_positions.reshape(-1)
        batch_size = X.shape[0]
        batch_idx = torch.arange(0, batch_size)
        # 假设batch_size=2，num_pred_positions=3
        # 那么batch_idx是np.array（[0,0,0,1,1]）
        # repeat_interleave类似于concat,可以按照dim(类似于concat的axis)进行不同维度的叠加
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions)
        masked_X = X[batch_idx, pred_positions]
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))
        mlm_Y_hat = self.mlp(masked_X)
        return mlm_Y_hat

In [8]:
mlm = MaskLM(vocab_size, num_hiddens)
mlm_positions = torch.tensor([[1, 5, 2], [6, 1, 5]])
# 放入神经网络进行训练
mlm_Y_hat = mlm(encoded_X, mlm_positions)
mlm_Y_hat.shape

torch.Size([2, 3, 10000])

通过掩码下的预测词元mlm_Y的真实标签mlm_Y_hat，我们可以计算在BERT预训练中的遮蔽语言模型任务的交叉熵损失。

In [9]:
mlm_Y = torch.tensor([[7, 8, 9], [10, 20, 30]])
loss = nn.CrossEntropyLoss(reduction='none')
mlm_l = loss(mlm_Y_hat.reshape((-1, vocab_size)), mlm_Y.reshape(-1))
mlm_l

tensor([9.5217, 9.4811, 9.2078, 9.4609, 8.8446, 8.8369],
       grad_fn=<NllLossBackward0>)

### 下一句预测（Next Sentence Prediction）

- 尽管掩蔽语言建模能够编码双向上下文来表示单词，但它不能显式地建模文本对之间的逻辑关系。
- 为了帮助理解两个文本序列之间的关系，BERT在预训练中考虑了一个二元分类任务——下一句预测。
- 在为预训练生成句子对时，有一半的时间它们确实是标签为“真”的连续句子；在另一半的时间里，第二个句子是从语料库中随机抽取的，标记为“假”。

- 下面的NextSentencePred类使用单隐藏层的多层感知机来预测第二个句子是否是BERT输入序列中第一个句子的下一个句子。
- 由于Transformer编码器中的自注意力，特殊词元$“<cls>”$的BERT表示已经对输入的两个句子进行了编码。
- 因此，多层感知机分类器的输出层（self.output）以X作为输入，其中X是多层感知机隐藏层的输出，而MLP隐藏层的输入是编码后的$“<cls>”$词元。

In [10]:
class NextSentencePred(nn.Module):
    """BERT的下一句预测任务"""
    def __init__(self, num_inputs, **kwargs):
        super(NextSentencePred, self).__init__(**kwargs)
        # 直接一个简单的二分类
        self.output = nn.Linear(num_inputs, 2)

    def forward(self, X):
        # X的形状：(batchsize,num_hiddens)
        return self.output(X)

In [11]:
encoded_X = torch.flatten(encoded_X, start_dim=1)
# NSP的输入形状:(batchsize，num_hiddens)
nsp = NextSentencePred(encoded_X.shape[-1])
nsp_Y_hat = nsp(encoded_X)
nsp_Y_hat.shape

torch.Size([2, 2])

### 整合代码

- 在预训练BERT时，最终的损失函数是掩蔽语言模型损失函数和下一句预测损失函数的线性组合。
- 现在我们可以通过实例化三个类BERTEncoder、MaskLM和NextSentencePred来定义BERTModel类。
- 前向推断返回编码后的BERT表示encoded_X、掩蔽语言模型预测mlm_Y_hat和下一句预测nsp_Y_hat。

In [12]:
class BERTModel(nn.Module):
    """BERT模型"""
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768,
                 hid_in_features=768, mlm_in_features=768,
                 nsp_in_features=768):
        super(BERTModel, self).__init__()
        # 前面的bert的encoder
        self.encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape,
                    ffn_num_input, ffn_num_hiddens, num_heads, num_layers,
                    dropout, max_len=max_len, key_size=key_size,
                    query_size=query_size, value_size=value_size)
        self.hidden = nn.Sequential(nn.Linear(hid_in_features, num_hiddens),
                                    nn.Tanh())
        # mask掩码矩阵
        self.mlm = MaskLM(vocab_size, num_hiddens, mlm_in_features)
        # 下一个句子预测
        self.nsp = NextSentencePred(nsp_in_features)

    def forward(self, tokens, segments, valid_lens=None,
                pred_positions=None):
        # bert主干
        encoded_X = self.encoder(tokens, segments, valid_lens)
        # 是否需要做位置掩码
        if pred_positions is not None:
            mlm_Y_hat = self.mlm(encoded_X, pred_positions)
        else:
            mlm_Y_hat = None
        # 用于下一句预测的多层感知机分类器的隐藏层，0是“<cls>”标记的索引
        # encoded_X维度tokens, segments, valid_lens, 这里取0那就是只要第一个句子
        nsp_Y_hat = self.nsp(self.hidden(encoded_X[:, 0, :]))
        return encoded_X, mlm_Y_hat, nsp_Y_hat